# Tarea 2
Estudiantes:
- Matías Fuentes
- Larry Uribne

Carga de librerías e inicialización de sparkContext

In [ ]:
from pyspark.sql import SparkSession
import math
spark = SparkSession.builder \
    .getOrCreate()
sc = spark.sparkContext

Ejemplo inicial

In [ ]:
nodes = [1, 2, 3, 4]
edges = [(1, 2), (2, 3), (2, 4), (3, 2)]
rdd_nodes = sc.parallelize(nodes)
rdd_edges = sc.parallelize(edges)

## Problema 1

1. Prepara un RDD que tenga cada nodo con su Page Rank inicial. Luego, haz una función que prepare el
mensaje que cada nodo va a enviar. Probablemente quieras almacenar estos valores como otro RDD.

Preparemos un RDD que contenga a cada nodo con su Page Rank inicial. El Page Rank inicial está dado por $\frac{1}{N_{nodos}}$

In [ ]:
n_nodes = rdd_nodes.count()
# Disponibilizamos la cantidad de nodos para todos los workers
bc_n_nodes = sc.broadcast(n_nodes)

rdd_ini = rdd_nodes.map(lambda x: (x, 1/bc_n_nodes.value))
rdd_ini.collect()

Para el paso siguiente necesitaremos la cantidad de vecinos

In [ ]:
neigh_counts = rdd_edges.countByKey()
# Disponibilizamos la cantidad de mensajes enviados por nodo para todos los workers
bc_neigh_counts = sc.broadcast(neigh_counts) 
bc_neigh_counts.value

El mensaje saliente de cada nodo será el mensaje inicial dividido por la cantidad de vecinos. Vemos que solo el nodo 2 tiene más de 1 vecino, por tanto el único mensaje que será modificado es el suyo.

In [ ]:
def prepare_node_msg(node, init_msg):
    if neigh_counts[node] != 0:
        message = init_msg / bc_neigh_counts.value[node]
        return message
    return init_msg

def preprare_rdd_msg(rdd_ini):
    rdd_prep_msg = rdd_ini.map(lambda x: (x[0], prepare_node_msg(x[0], x[1])))
    return rdd_prep_msg

rdd_prep_msg = preprare_rdd_msg(rdd_ini)
rdd_prep_msg.collect()

2. Función que envía los mensajes a los nodos correspondientes y se hace cargo del merge de los mensajes recibidos por cada nodo. Debe retornar un RDD que para cada nodo diga cuál es el mensaje final recibido.

Primero hacemos un join. Las llaves son los nodos emisores, y los valores resultantes serán tuplas donde el primer elemento es el nodo receptor y el segundo elemento es el mensaje recibido.

Luego, generamos otro RDD que contenga solo los valores de la operación anterior y aplicamos reduceByKey para sumar los mensajes, obteniendo así tuplas donde la llave es el nodo receptor y el valor es la suma de todos los mensajes recibidos.

In [ ]:
def send_msg(rdd_edges, rdd_prep_msg):
    rdd_msg_sent = rdd_edges.join(rdd_prep_msg)
    rdd_received_msg = rdd_msg_sent.values().reduceByKey(lambda x, y: x + y)
    return rdd_received_msg
rdd_received_msg = send_msg(rdd_edges, rdd_prep_msg)
rdd_received_msg.collect()

3. Haz una función que actualice el valor de Page Rank para cada nodo considerando el damping factor.
Probablemente quieras hacer una función que tome el output del punto anterior y lo procese.

In [ ]:
# Disponibilizamos el damping factor para todos los workers
damping = 0.85
bc_damping = sc.broadcast(damping)
def update_pr(rdd_received_msg):
    rdd_updated_pr = rdd_received_msg.mapValues(lambda x: x * bc_damping.value + (1-bc_damping.value)/bc_n_nodes.value)
    return rdd_updated_pr
rdd_updated_pr = update_pr(rdd_received_msg)
rdd_updated_pr.collect()

4. Itera los pasos correspondientes por un número máximo de iteraciones, o hasta que la diferencia entre dos iteraciones del valor de Page Rank sea mínima.

Para facilitar la legibilidad, consolidemos el proceso completo en una función. Usaremos el error absoluto medio como criterio de parada 

In [ ]:
def mean_absolute_error(rdd1, rdd2):
    abs_difference = rdd1.join(rdd2).values().map(lambda x: abs(x[0] - x[1]))
    mean_abs_difference = abs_difference.reduce(lambda x, y: x + y)/2
    return mean_abs_difference

def page_rank(rdd_nodes, rdd_edges, damping=0.85, max_iterations = 10, eps=0.05):
    n_nodes = rdd_nodes.count()
    # Disponibilizamos la cantidad de nodos para todos los workers
    bc_n_nodes = sc.broadcast(n_nodes)
    neigh_counts = rdd_edges.countByKey()
    # Disponibilizamos la cantidad de mensajes enviados por nodo para todos los workers
    bc_neigh_counts = sc.broadcast(neigh_counts) 
    bc_neigh_counts.value
    # Disponibilizamos el damping factor para todos los workers
    bc_damping = sc.broadcast(damping)
    # PageRank inicial
    rdd_ini = rdd_nodes.map(lambda x: (x, 1/bc_n_nodes.value))
    prev_pr = rdd_ini
    # Preparar mensaje inicial
    rdd_prep_msg = preprare_rdd_msg(rdd_ini)
    
    for i in range(max_iterations):
        # Enviar mensaje y gestionar merge al recibir
        rdd_received_msg = send_msg(rdd_edges, rdd_prep_msg)
        # Actualizar PageRank
        rdd_updated_pr = update_pr(rdd_received_msg)
        # Calcular distancia entre el PageRank recién obtenido y el de la iter previa
        mean_distance = mean_absolute_error(prev_pr, rdd_updated_pr)
        # Condición de parada
        if mean_distance < eps:
            break
        # Mensaje enviado para iteración siguiente
        rdd_prep_msg = preprare_rdd_msg(rdd_updated_pr)
        # El PageRank de ahora es el PageRank previo en la próxima iteración
        prev_pr = rdd_updated_pr
        
    print(f'Total iterations: {i}')
    return rdd_updated_pr

In [ ]:
final_pr = page_rank(rdd_nodes, rdd_edges)
final_pr.collect()

### Prueba con Cora

In [ ]:
import pandas as pd

In [ ]:
citas = pd.read_csv('cora/cora.cites',sep="\t",
                    header=None,
                    names=["target", "source"])

In [ ]:
nodes = list(set(list(citas.target.values) + list(citas.source.values)))
edges = []
for source, target in citas.values:
    edges.append((source, target))
rdd_nodes = sc.parallelize(nodes)
rdd_edges = sc.parallelize(edges)

In [ ]:
rdd_result = page_rank(rdd_nodes, rdd_edges)

In [ ]:
rdd_result.collect()

## Problema 2

Recibir input

In [184]:
nodes = list(range(1, 9))
edges = [(1,2,4), (1,3,2), (1,4,1), (3,2,1), (3,5,6),(4,5,9),(4,6,2),(7,8,9), (5,6,1)]

Generar RDDs

In [185]:
rdd_nodes = sc.parallelize(nodes)
rdd_edges = sc.parallelize(edges)

Formateamos las aristas

In [186]:
rdd_edges = rdd_edges.map(lambda x: (x[0], (x[1], x[2])))
rdd_edges.collect()

[(1, (2, 4)),
 (1, (3, 2)),
 (1, (4, 1)),
 (3, (2, 1)),
 (3, (5, 6)),
 (4, (5, 9)),
 (4, (6, 2)),
 (7, (8, 9)),
 (5, (6, 1))]

1. Escoge el nodo inicial, este nodo tiene costo acumulado 0 y todos los demás tienen costo acumulado
infinito.

In [188]:
def init_rdd(rdd_nodes, initial_node):
    initial_node = initial_node
    bc_initial_node = sc.broadcast(initial_node) # Disponibilizar para todos los worker
    return rdd_nodes.map(lambda x: (x, math.inf) if x != bc_initial_node.value else (x, 0))
rdd_init = init_rdd(rdd_nodes, 1)
rdd_init.collect()

[(1, 0), (2, inf), (3, inf), (4, inf), (5, inf), (6, inf), (7, inf), (8, inf)]

2. En cada iteración, cada nodo comunica el costo acumulado a sus vecinos. Cada nodo recibe este costo,
sumado con el costo de atravesar la arista.

In [ ]:
rdd_edges.join(rdd_init).collect()

In [ ]:
def pass_msg(rdd_edges, rdd_prev):
    return rdd_edges.join(rdd_prev).mapValues(lambda x: (x[0][0], x[0][1] + x[1])).values()
rdd_pass_msg = pass_msg(rdd_edges, rdd_init)
rdd_pass_msg.collect()

3. Para hacer merge de todos los mensajes dejamos el mínimo de todos los costos. Así, actualizamos cada
nodo con el costo mínimo recibido solo si es menor al costo acumulado que ya tenía ese nodo.

In [ ]:
def reduce_msg(rdd_pass_msg):
    return rdd_pass_msg.reduceByKey(lambda x, y: min(x, y))
    
rdd_reduce_msg = reduce_msg(rdd_pass_msg)
rdd_reduce_msg.collect()

In [ ]:
def robust_min(x):
    a, b = x
    try:
        return min(a,b)
    except:
        return a if a != None else b
    
def update_cost(rdd_prev, rdd_reduce_msg):
    return rdd_prev.leftOuterJoin(rdd_reduce_msg).mapValues(lambda x: robust_min(x))

rdd_updated_cost = update_cost(rdd_init, rdd_reduce_msg)
rdd_updated_cost.collect()

4. Si en dos iteraciones el costo en llegar para cada nodo no cambia, entonces nos detenemos.

In [ ]:
def stop_condition(rdd_pre, rdd_post):
    # RDD donde las llaves son los nodos y los valores son 1 si no hubo cambio y 0 si lo hubo
    rdd_track_changes = rdd_pre.join(rdd_post).mapValues(lambda x: int(x[0] == x[1]))
    # Intersección
    n_changes = rdd_track_changes.values().reduce(lambda x, y: x + y) 
    return n_changes == 0
stop_condition(rdd_init, rdd_updated_cost)

Consolidar en una función

In [189]:
def init_rdd(rdd_nodes, initial_node):
    bc_initial_node = sc.broadcast(initial_node) # Disponibilizar para todos los worker
    return rdd_nodes.map(lambda x: (x, math.inf) if x != bc_initial_node.value else (x, 0))

def pass_msg(rdd_edges, rdd_prev):
    return rdd_edges.join(rdd_prev).mapValues(lambda x: (x[0][0], x[0][1] + x[1])).values()

def reduce_msg(rdd_pass_msg):
    return rdd_pass_msg.reduceByKey(lambda x, y: min(x, y))

def robust_min(x):
    a, b = x
    try:
        return min(a,b)
    except:
        return a if a != None else b
    
def update_cost(rdd_prev, rdd_reduce_msg):
    return rdd_prev.leftOuterJoin(rdd_reduce_msg).mapValues(lambda x: robust_min(x))
    
def stop_condition(rdd_pre, rdd_post):
    # RDD donde las llaves son los nodos y los valores son 0 si no hubo cambio y 1 si lo hubo
    rdd_track_changes = rdd_pre.join(rdd_post).mapValues(lambda x: int(x[0] != x[1]))
    
    # Intersección
    n_changes = rdd_track_changes.values().reduce(lambda x, y: x + y) 
    return n_changes == 0

def single_source_shortest_path(rdd_nodes, rdd_edges, initial_node, max_iterations = 10000):
    rdd_edges = rdd_edges.map(lambda x: (x[0], (x[1], x[2])))

    # Nodo inicial en 0, el resto en infinito
    rdd_init = init_rdd(rdd_nodes,initial_node)
    rdd_prev_cost = rdd_init
    
    stop_counter = 0
    for i in range(max_iterations):
        rdd_pass_msg = pass_msg(rdd_edges, rdd_prev_cost)
        #rdd_pass_msg = rdd_edges.join(rdd_prev_cost).mapValues(lambda x: (x[0][0], x[0][1] + x[1])).values()
        rdd_reduce_msg = reduce_msg(rdd_pass_msg)
        # rdd_reduce_msg = rdd_pass_msg.reduceByKey(lambda x, y: min(x, y))
        rdd_updated_cost = update_cost(rdd_prev_cost, rdd_reduce_msg)
        #rdd_updated_cost = rdd_prev_cost.leftOuterJoin(rdd_reduce_msg).mapValues(lambda x: robust_min(x))
        if stop_condition(rdd_prev_cost, rdd_updated_cost):
            stop_counter += 1
            if stop_counter == 2:
                break
        else:
            stop_counter = 0 
        rdd_prev_cost = rdd_updated_cost
        #print("Iteracion:", i)
        #print(rdd_updated_cost.collect())
    return rdd_updated_cost


In [ ]:
nodes = list(range(1, 9))
edges = [(1,2,4), (1,3,2), (1,4,1), (3,2,1), (3,5,6),(4,5,9),(4,6,2),(7,8,9), (5,6,1)]

initial_node = 1
rdd_nodes = sc.parallelize(nodes)
rdd_edges = sc.parallelize(edges)
rdd_result = single_source_shortest_path(rdd_nodes, rdd_edges, initial_node)

In [ ]:
rdd_result.collect()

### Pruebas con Cora

In [ ]:
nodes = list(set(list(citas.target.values) + list(citas.source.values)))
edges = []
for target, source in citas.values:
    edges.append((source, target, 1))
rdd_nodes = sc.parallelize(nodes)
rdd_edges = sc.parallelize(edges)
initial_node = 1105932
rdd_result = single_source_shortest_path(rdd_nodes, rdd_edges, initial_node)

ERROR:root:KeyboardInterrupt while sending command.>          (6473 + 8) / 8192]
Traceback (most recent call last):
  File "/Users/matif/.pyenv/versions/3.10.0/envs/tareas_proce/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/matif/.pyenv/versions/3.10.0/envs/tareas_proce/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/matif/.pyenv/versions/3.10.0/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
result = rdd_result.collect()
for destination, cost in result:
    if cost < math.inf:
        print(destination, cost)

In [179]:
result

[(155736, inf),
 (1106112, inf),
 (1130808, inf),
 (1114512, inf),
 (504, inf),
 (90888, inf),
 (1131312, inf),
 (566664, inf),
 (3192, inf),
 (77112, inf),
 (593544, inf),
 (397488, inf),
 (94416, inf),
 (1119216, inf),
 (595056, inf),
 (644448, inf),
 (46536, inf),
 (71736, inf),
 (6216, inf),
 (71904, inf),
 (56112, inf),
 (41216, inf),
 (18536, inf),
 (272720, inf),
 (1134056, inf),
 (1117760, inf),
 (1125992, inf),
 (200480, inf),
 (135464, inf),
 (5600, inf),
 (32872, inf),
 (616336, inf),
 (379288, inf),
 (739816, inf),
 (469504, inf),
 (28336, inf),
 (28504, inf),
 (1110256, inf),
 (561568, inf),
 (751408, inf),
 (1153264, inf),
 (1121176, inf),
 (1130080, inf),
 (573553, inf),
 (18313, inf),
 (92065, inf),
 (166825, inf),
 (44017, inf),
 (928873, inf),
 (593209, inf),
 (110041, inf),
 (1118209, inf),
 (20833, inf),
 (643777, inf),
 (1135345, inf),
 (62329, inf),
 (6217, inf),
 (6385, inf),
 (334153, inf),
 (1112665, inf),
 (1104769, inf),
 (1121569, inf),
 (8961, inf),
 (64836

In [180]:
nodes_in_big_c = [1122304, 1155073, 851968, 1114118, 1105932, 1114125, 253971, 8213, 696342, 696343, 696345, 696346, 8224, 950305, 262178, 35, 630817, 40, 1114153, 1130539, 385067, 131117, 131122, 16437, 188471, 16451, 1138755, 1130567, 1130568, 1114184, 16461, 1114192, 16470, 16471, 155736, 16474, 1130586, 16476, 155738, 106590, 573535, 589923, 16485, 1130600, 32872, 630890, 1114222, 573553, 65650, 114, 65653, 117, 221302, 270456, 1122425, 1114239, 128, 130, 1106052, 1130634, 1130637, 647315, 180373, 1122460, 1130653, 1130657, 164, 82087, 82090, 180399, 82098, 1130676, 1130678, 1106103, 1130680, 1106112, 590022, 1114331, 1114336, 385251, 631015, 688361, 647408, 1114352, 131315, 647413, 33013, 131318, 131317, 1106172, 1114364, 213246, 41216, 229635, 270600, 631052, 1122574, 606479, 1114388, 1122580, 647447, 1138968, 1138970, 139547, 1130780, 1114398, 213279, 288, 1130808, 1106236, 672064, 672070, 672071, 1114442, 49482, 1122642, 1130847, 90470, 1130856, 1106287, 1106298, 98693, 1114502, 24966, 8581, 205192, 98698, 205196, 24974, 1114512, 1122704, 1106330, 1114526, 147870, 1130915, 57764, 424, 8617, 8619, 1130927, 1130929, 434, 1130931, 16819, 1130934, 606647, 467383, 197054, 1106370, 459206, 41417, 16843, 459213, 459214, 463, 459216, 33231, 1106388, 377303, 139738, 1106401, 1106406, 1114605, 8687, 1106418, 8696, 504, 506, 8699, 49660, 582139, 8703, 1114629, 573964, 238099, 33301, 33303, 573978, 90655, 385572, 1114664, 33325, 574009, 1106492, 57922, 787016, 1000012, 57932, 139865, 57948, 25181, 25184, 1131116, 1106546, 1106547, 8821, 8832, 1131137, 33412, 1106568, 1131149, 1131150, 49811, 1114777, 1131164, 1131165, 1131167, 8865, 123556, 1131172, 8872, 107177, 8874, 8875, 1131180, 656048, 328370, 49843, 49844, 1131189, 49847, 1131192, 74427, 1131198, 41666, 1106630, 582343, 148170, 950986, 582349, 975567, 99023, 688849, 99025, 1114838, 99030, 1131223, 1131230, 1131236, 140005, 49895, 82664, 82666, 1106671, 1114864, 41714, 107251, 107252, 1131257, 1131258, 1123068, 8961, 1131266, 1131267, 41732, 1131270, 90888, 1131274, 1131277, 1123087, 1123093, 1131300, 1131301, 1131305, 1131312, 17201, 1131314, 574264, 17208, 238401, 1131334, 1131335, 197452, 1106764, 1131345, 1106771, 1131348, 246618, 17242, 1131359, 1131360, 1106789, 656231, 1131374, 582511, 1114992, 1123188, 148341, 887, 906, 910, 230300, 263069, 91038, 1106849, 1106854, 1123239, 936, 940, 941, 943, 148399, 123825, 1131464, 1131466, 254923, 74698, 74700, 1131471, 17363, 82920, 943087, 66556, 74749, 574462, 689152, 1026, 66563, 66564, 1033, 1034, 1035, 164885, 1106966, 33818, 1131549, 1131550, 33823, 1115166, 66594, 66596, 1131557, 648232, 287787, 1131565, 107569, 1107010, 17476, 17477, 58436, 74821, 17488, 58453, 58454, 1131607, 312409, 1131611, 1107041, 762980, 25702, 33895, 263279, 33904, 1131634, 33907, 1107062, 1131639, 853114, 1107067, 853115, 853116, 853118, 1131647, 156794, 763009, 763010, 189571, 189572, 189566, 189574, 189577, 197783, 1107095, 1115291, 853150, 124064, 50336, 50337, 853155, 591016, 591017, 58540, 25772, 42156, 50354, 189620, 189623, 58552, 25791, 1107136, 66751, 25794, 1107140, 1123530, 25805, 50381, 1131728, 1237, 1131734, 189655, 1139928, 566488, 1131741, 1246, 74975, 42207, 1123553, 1131745, 1107171, 1131748, 66782, 42209, 1131752, 66794, 42221, 1115375, 66805, 574710, 1272, 66809, 1123576, 345340, 255233, 353541, 189708, 1107215, 189721, 140569, 9513, 9515, 763181, 509233, 156977, 1131828, 116021, 263482, 263486, 1115456, 124224, 1140040, 263498, 189774, 1115471, 1365, 1385, 1123689, 9581, 1107312, 116081, 9586, 75121, 116084, 1107319, 116087, 1107325, 566653, 411005, 263553, 509315, 58758, 17798, 124296, 566664, 755082, 17811, 1107355, 976284, 189856, 66982, 1107367, 66986, 1123756, 66990, 206259, 820662, 1107385, 91581, 509379, 214472, 1481, 976334, 34257, 411092, 34263, 1107418, 34266, 230879, 714208, 230884, 9708, 9716, 83449, 1107455, 83461, 1140230, 1140231, 34315, 108047, 714256, 755217, 714260, 181782, 1115670, 1123867, 1115677, 206371, 714289, 34355, 1132083, 1115701, 75318, 1140289, 427606, 1123926, 575077, 1107558, 640617, 1107567, 1107572, 1132157, 738941, 255628, 1115790, 583318, 50838, 1688, 1123991, 1694, 59045, 67245, 67246, 1717, 206524, 370366, 132806, 91852, 91853, 354004, 132821, 157401, 1107674, 67292, 567005, 567018, 1115886, 1786, 1132285, 83725, 1817, 231198, 444191, 83746, 173863, 198443, 116528, 1115959, 575292, 173884, 124734, 1140543, 116545, 395075, 1140547, 1140548, 91975, 116552, 116553, 18251, 182093, 182094, 444240, 231249, 67415, 911198, 42847, 42848, 1132385, 575331, 100197, 51045, 337766, 141160, 51049, 59244, 51052, 83826, 141171, 1919, 1132418, 83847, 18313, 1116044, 1132434, 34708, 1107861, 75674, 1132443, 124828, 1949, 1951, 1952, 92065, 1953, 1955, 1956, 1959, 575402, 75691, 1132459, 1132461, 75693, 75695, 75694, 10169, 608190, 608191, 10174, 10177, 1132486, 10183, 10186, 1997, 1999, 1132505, 272345, 632796, 51180, 1116146, 714748, 198653, 67584, 141324, 1116181, 124952, 141342, 141347, 608292, 632874, 67633, 84020, 84021, 116790, 1050679, 157761, 608326, 1108050, 18532, 632935, 18536, 256106, 1116268, 157805, 714879, 346243, 34961, 18582, 264347, 1132706, 34979, 149669, 1116328, 1116336, 43186, 346292, 18615, 239800, 1116347, 1132731, 18619, 10430, 75969, 239810, 10435, 75972, 1108167, 1108169, 75983, 1108175, 198866, 239829, 714975, 26850, 190697, 190698, 59626, 1116397, 1108209, 190706, 35061, 633081, 1116410, 35070, 1132815, 395540, 395547, 141596, 395553, 10531, 1108267, 280876, 2354, 575795, 248119, 289085, 289088, 1132864, 59715, 321861, 919885, 272720, 174418, 18770, 18773, 18774, 1132887, 18777, 174425, 18781, 756061, 100701, 18785, 1108329, 264556, 1116530, 1132922, 18811, 59772, 739707, 18812, 18815, 2440, 1108363, 18832, 18833, 18834, 158098, 1132948, 59798, 379288, 1116569, 854434, 108962, 330148, 1108389, 108963, 1132968, 92589, 133550, 362926, 108974, 133553, 1116594, 108983, 133563, 133566, 133567, 1133004, 395725, 1133010, 1116629, 158172, 330208, 1133028, 739816, 84459, 1124844, 133615, 1133047, 469504, 649730, 649731, 35335, 649739, 35343, 68115, 166420, 207395, 27174, 10793, 10796, 141868, 10798, 436796, 27199, 428610, 117315, 117316, 27203, 100935, 1108551, 248395, 117328, 2653, 27230, 2654, 100961, 2658, 19045, 2663, 248425, 27241, 2665, 27243, 27246, 248431, 27249, 27250, 1108597, 51831, 43639, 51834, 68224, 2695, 2696, 2698, 1133196, 2702, 51866, 35490, 1116835, 51879, 1116839, 1116842, 682666, 1108656, 43698, 559804, 240321, 51909, 387795, 84695, 1125082, 51934, 1125092, 10981, 568045, 191216, 633585, 191222, 1108728, 1116922, 510715, 510718, 60159, 273152, 576257, 60169, 60170, 109323, 101143, 101145, 1133338, 19231, 502574, 1116974, 682815, 1133390, 11093, 1108834, 215912, 1108841, 576362, 1133417, 68463, 1133428, 27510, 633721, 27514, 27530, 27531, 101261, 27535, 101263, 68495, 199571, 158614, 27543, 68505, 1133469, 1117089, 641956, 166825, 191404, 134060, 641976, 592826, 142268, 592830, 35778, 35797, 27606, 27612, 27623, 27627, 27631, 134128, 27632, 44017, 265203, 289779, 289780, 289781, 248823, 1117184, 1125386, 35852, 3084, 3085, 35854, 1125393, 35863, 3095, 1109017, 3097, 1125402, 3101, 1117219, 3112, 134199, 11325, 11326, 35905, 1117249, 11335, 11337, 134219, 11339, 592973, 11342, 592975, 166989, 35922, 93273, 592986, 1125467, 158812, 289885, 1125469, 314459, 592993, 592996, 928873, 1059953, 3187, 1125492, 3191, 3192, 396412, 593022, 93318, 93320, 3217, 3218, 3220, 3222, 240791, 175256, 3223, 289945, 519318, 3229, 3231, 3232, 3233, 3235, 593060, 19621, 3237, 3236, 3240, 1117348, 134307, 3243, 593068, 134316, 134314, 134315, 126128, 134320, 576691, 519353, 175291, 1109185, 593091, 1109199, 593104, 593105, 576725, 1109208, 1125597, 19697, 27895, 593155, 60682, 576795, 101660, 101662, 52515, 1117476, 36131, 36140, 593201, 36145, 85299, 77108, 77112, 593209, 593210, 1117501, 118079, 36162, 36167, 85324, 44368, 593240, 683355, 593248, 683360, 85352, 593260, 93555, 601462, 28026, 683404, 1109392, 44455, 593328, 593329, 1117618, 101811, 486840, 175548, 1109439, 1134022, 85449, 85452, 576973, 1117653, 175576, 110041, 601561, 208345, 601567, 44514, 241133, 470511, 445938, 118259, 118260, 1125895, 1125906, 552469, 1125909, 568857, 273949, 1109542, 28202, 52784, 650807, 323128, 1125944, 93755, 642621, 650814, 577086, 1117760, 1125953, 1109566, 28227, 28230, 249421, 1109581, 69198, 642641, 650834, 110163, 110162, 110164, 28249, 1117786, 28254, 52835, 1125992, 28265, 1125993, 28267, 52847, 28278, 642681, 1126011, 1126012, 77438, 28287, 28290, 593544, 1117833, 61069, 1126029, 61073, 233106, 1126037, 593559, 593560, 118424, 1126044, 1126050, 118435, 69284, 118436, 102061, 69296, 28336, 85688, 28350, 28359, 577227, 77515, 20178, 20179, 20180, 429781, 1117920, 20193, 28385, 93923, 28387, 28389, 167656, 429805, 642798, 1134320, 3828, 1117942, 167670, 28412, 1134346, 642827, 36620, 1134348, 69392, 69397, 118558, 28447, 200480, 118559, 642847, 175909, 28456, 560936, 69418, 216877, 216878, 577331, 28471, 28473, 446271, 1118017, 675649, 28485, 1109830, 307015, 28487, 28489, 126793, 28491, 642894, 28504, 3932, 184157, 1109873, 12155, 12158, 28542, 61312, 315266, 1109891, 1118083, 12165, 151430, 12169, 1118092, 593813, 12182, 118682, 12194, 12195, 12197, 12198, 12199, 1118120, 675756, 12210, 12211, 200630, 643003, 126909, 77758, 126912, 36802, 593859, 1109957, 126920, 954315, 12238, 1126350, 126926, 126927, 12247, 28632, 135130, 143323, 28640, 28641, 815073, 61417, 28649, 733167, 1110000, 12275, 815096, 45052, 643069, 593921, 1118209, 249858, 77826, 77829, 102406, 675847, 45061, 1110024, 28674, 1110028, 94229, 634902, 593942, 634904, 217115, 1118245, 12330, 20526, 20528, 12337, 217139, 987188, 20534, 127033, 634938, 12347, 987197, 12350, 503871, 503877, 12359, 503883, 282700, 1118286, 503893, 561238, 118873, 594011, 1118302, 634975, 1126503, 20584, 594025, 340075, 340078, 20592, 20593, 143476, 594039, 20601, 20602, 1118332, 372862, 594047, 643199, 45188, 45189, 307336, 1118347]

max_val = 0
for initial_node in nodes_in_big_c:
    counter = 0
    for source, target, weight in edges:
        if source == initial_node:
            counter +=1
    if counter > max_val:
        final_node = initial_node
        max_val = counter

final_node, max_val
    

(1105932, 5)